## Data Visualization Course | VNU-HCM University of Science.
### Final Project
#### **STEP 02 - DATA Preprocessing**

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from Libraries import *
from Shared_Functions import *

In [3]:
dataset_source_path = getGranDir() / 'Dataset'
pre_process_dataset_path = dataset_source_path / '2021.csv'
raw_df = pd.read_csv(pre_process_dataset_path, index_col=0)
raw_df.head()

,id,year,mathematics_score,literature_score,physics_score,chemistry_score,biology_score,foreign_language_score,foreign_language_type,english_score,history_score,geography_score,civic_education_score,chinese_score,japanese_score,korean_score,french_score,german_score,russian_score
0,1000046,2022,8.0,6.25,7.25,5.5,4.75,4.6,N1,4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000031,2022,7.0,7.75,NaN,NaN,NaN,8.8,N1,8.8,6.25,5.0,9.00,NaN,NaN,NaN,NaN,NaN,NaN
2,1000012,2022,6.4,7.50,NaN,NaN,NaN,NaN,NaN,NaN,4.75,5.0,8.00,NaN,NaN,NaN,NaN,NaN,NaN
3,1000087,2022,8.4,8.00,NaN,NaN,NaN,9.4,N1,9.4,6.75,6.5,8.25,NaN,NaN,NaN,NaN,NaN,NaN
4,1000071,2022,7.6,8.25,6.25,5.0,3.25,6.6,N1,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**1. Kiểm tra kiểu dữ liệu và số lượng hàng, cột**

In [4]:
print(raw_df.dtypes)
raw_df.shape

id                          int64
year                        int64
mathematics_score         float64
literature_score          float64
physics_score             float64
chemistry_score           float64
biology_score             float64
foreign_language_score    float64
foreign_language_type      object
english_score             float64
history_score             float64
geography_score           float64
civic_education_score     float64
chinese_score             float64
japanese_score            float64
korean_score              float64
french_score              float64
german_score              float64
russian_score             float64
dtype: object


(995381, 19)

**Nhận xét:** Các cột đã có kiểu dữ liệu phù hợp với ý nghĩa.

**2. Kiểm tra điều kiện tồn tại của các cột.**

**2.1 Kiểm tra cột ID: các ID này phải là DUY NHẤT, và phải lớn hơn 0.**

In [5]:
# ID
ID_col = raw_df['id']
less_than_0 = ID_col[ID_col < 0]
duplicate_values = ID_col[ID_col.duplicated(keep=False)]
print('Less than zero: ')
print(less_than_0)
print('Values is not unique: ')
print(duplicate_values)

Less than zero: 
Series([], Name: id, dtype: int64)
Values is not unique: 
Series([], Name: id, dtype: int64)


**Nhận xét:** Như vậy không có giá trị nào bé hơn không và không có giá trị nào bị trùng lập.

**Kết luận:** Vì không có trùng lặp cho nên không có hàng nào trong dataset sẽ bị trùng lập.

**2.2 Kiểm tra cột Year: Year phải lớn hơn 0 và phải bằng 2022.**

In [6]:
#Year
Year_col = raw_df['year']
less_than_0 = Year_col[Year_col < 0]
is_not_2022 = Year_col[Year_col != 2022]
print('Less than zero: ')
print(less_than_0)
print('Value must be 2022: ')
print(is_not_2022)

Less than zero: 
Series([], Name: year, dtype: int64)
Value must be 2022: 
Series([], Name: year, dtype: int64)


**Nhận xét:** Như vậy không có giá trị nào khác 2022 và bé hơn 0.
**Kết luận:** Vì không có giá trị nào khác 2022 cho nên việc phân tích cột này sẽ không có ý nghĩa. Do đó ta sẽ bỏ cột này luôn.

In [7]:
raw_df = raw_df.drop(columns='year')
raw_df.head()

,id,mathematics_score,literature_score,physics_score,chemistry_score,biology_score,foreign_language_score,foreign_language_type,english_score,history_score,geography_score,civic_education_score,chinese_score,japanese_score,korean_score,french_score,german_score,russian_score
0,1000046,8.0,6.25,7.25,5.5,4.75,4.6,N1,4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000031,7.0,7.75,NaN,NaN,NaN,8.8,N1,8.8,6.25,5.0,9.00,NaN,NaN,NaN,NaN,NaN,NaN
2,1000012,6.4,7.50,NaN,NaN,NaN,NaN,NaN,NaN,4.75,5.0,8.00,NaN,NaN,NaN,NaN,NaN,NaN
3,1000087,8.4,8.00,NaN,NaN,NaN,9.4,N1,9.4,6.75,6.5,8.25,NaN,NaN,NaN,NaN,NaN,NaN
4,1000071,7.6,8.25,6.25,5.0,3.25,6.6,N1,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**2.3 Kiểm tra các cột điểm - score: Score phải >= 0 và <= 10 hoặc bằng NaN.**

In [8]:
score_df = raw_df.select_dtypes(include='float64')
score_df.head()
for col in score_df.columns:
    less_than_0 = score_df[score_df[col] < 0]
    greater_than_10 = score_df[score_df[col] > 10]
    
    if len(less_than_0.index.tolist()):
        print(f'Column: {col}')
        print('Less than zero (indices): ')
        print(less_than_0.index.tolist())
    
    if len(greater_than_10.index.tolist()):
        print(f'Column: {col}')
        print('Greater than ten (indices): ')
        print(greater_than_10.index.tolist())

**Nhận xét:** Không có output được in ra, tức nghĩa là các cột đều đã thõa mãn điều kiện.

**2.4 Kiểm tra cột có kiểu dữ liệu không phải dữ liệu kiểu số**

Ở đây ta chỉ cần tìm ra các trường DUY NHẤT.

In [9]:
non_numerical = raw_df['foreign_language_type']
unique_values = non_numerical.unique()
unique_values

array(['N1', nan, 'N4', 'N6', 'N7', 'N3', 'N5', 'N2'], dtype=object)

**3. Kiểm tra các thiếu khuyết trong dữ liệu:**

In [10]:
def calcMissingRatio(df):
    missing_ratio = {}
    for key, values in df.items():
        missing_count = values.isna().sum()
        total_count = len(values)
        missing_ratio[key] = round(( missing_count / total_count )* 100, 1) if total_count != 0 else None
    return missing_ratio

In [11]:
missing_ratio = calcMissingRatio(raw_df)
for key, values in missing_ratio.items():
    print(f'Column: {key} - Missing Ratio: {values}')

Column: id - Missing Ratio: 0.0
Column: mathematics_score - Missing Ratio: 1.3
Column: literature_score - Missing Ratio: 1.4
Column: physics_score - Missing Ratio: 67.3
Column: chemistry_score - Missing Ratio: 67.1
Column: biology_score - Missing Ratio: 67.6
Column: foreign_language_score - Missing Ratio: 12.5
Column: foreign_language_type - Missing Ratio: 12.5
Column: english_score - Missing Ratio: 13.0
Column: history_score - Missing Ratio: 33.7
Column: geography_score - Missing Ratio: 34.0
Column: civic_education_score - Missing Ratio: 44.3
Column: chinese_score - Missing Ratio: 99.8
Column: japanese_score - Missing Ratio: 99.9
Column: korean_score - Missing Ratio: 100.0
Column: french_score - Missing Ratio: 99.9
Column: german_score - Missing Ratio: 100.0
Column: russian_score - Missing Ratio: 100.0


**Nhận xét:**
1. Ta thấy các cột như: `russian_score`, `german_score`, `french_score`, `korean_score`, `japanese_score`, `chinese_score` đều có tỉ lệ thiếu khuyết lên đến hơn 99.9%. Do đó sẽ không có quá nhiều ý nghĩa phân tích. 
2. Các cột như `id` và `year` có tỉ lệ thiếu khuyết là 0.0%.

**Kết luận:** 
1. Ta sẽ chỉ xét các môn thi chính thức như toán - lý - hóa - sinh - văn - anh văn - sử - địa - GDCD. 
2. Do đó ta sẽ tiến hành loại bỏ các cột không có ý nghĩa phân tích: `russian_score`, `german_score`, `french_score`, `korean_score`, `japanese_score`, `chinese_score`, `foreign_language_score`.
3. Vì do chúng ta đã xóa các cột ngoại ngữ khác vì tỉ lệ thiếu khuyết cao thì ta sẽ xóa luôn cột định nghĩa ký hiệu cho các loại ngoại ngữ : `foreign_language_type`

In [12]:
raw_df = raw_df.drop(columns = ['russian_score', 'german_score', 'french_score', 'korean_score', 'japanese_score', 'chinese_score', 'foreign_language_score', 'foreign_language_type'])
raw_df.head()

,id,mathematics_score,literature_score,physics_score,chemistry_score,biology_score,english_score,history_score,geography_score,civic_education_score
0,1000046,8.0,6.25,7.25,5.5,4.75,4.6,NaN,NaN,NaN
1,1000031,7.0,7.75,NaN,NaN,NaN,8.8,6.25,5.0,9.00
2,1000012,6.4,7.50,NaN,NaN,NaN,NaN,4.75,5.0,8.00
3,1000087,8.4,8.00,NaN,NaN,NaN,9.4,6.75,6.5,8.25
4,1000071,7.6,8.25,6.25,5.0,3.25,6.6,NaN,NaN,NaN


**4. Có hai khối thi là Khoa học tự nhiên và Khoa học xã hội, khi ta thi KHTN ( lý - hóa - sinh ) thì sẽ không thi các môn KHXH ( sử - địa - GDCD ).**

Chính vì vậy mà ta sẽ không tiến hành điền khuyết NaN vào trong các trường dữ liệu này. Mà ta sẽ phân tích dựa theo hai tổ hợp môn này.

**5. Sắp xếp các kết quả hàng theo id.**

In [16]:
raw_df.sort_values(by='id', ascending=True, inplace=True)
raw_df.head()

,id,mathematics_score,literature_score,physics_score,chemistry_score,biology_score,english_score,history_score,geography_score,civic_education_score
80,1000001,3.6,5.00,NaN,NaN,NaN,4.0,2.75,6.0,8.75
68,1000002,8.4,6.75,NaN,NaN,NaN,7.6,8.50,7.5,8.25
77,1000003,5.8,7.50,NaN,NaN,NaN,5.0,7.25,5.5,8.75
42,1000004,7.4,7.50,NaN,NaN,NaN,8.6,7.50,6.5,7.50
41,1000005,7.2,8.50,NaN,NaN,NaN,9.0,8.00,8.5,8.25


**6. Tiến hành ghi tập dữ liệu sau khi xử lý vào file csv.**

In [17]:
processed_dataset_path = dataset_source_path / 'THPTQG_2022_processed.csv'
raw_df.to_csv(processed_dataset_path, index=False)